In [1]:
import sys

sys.path.append("/home/weber/PycharmProjects/EXOTIC/src")
from utils.utils import load_config_file, mkdir

yaml = load_config_file(config_file="/home/weber/PycharmProjects/EXOTIC/clean/src/config_clean_clean.yaml")


In [2]:
import hail as hl
hl.init(min_block_size=128)
import os
import pandas as pd
import subprocess
os.environ['PYSPARK_SUBMIT_ARGS'] = "--driver-memory 200 pyspark-shell"
# !module load java/java-1.8
# gs = "/gstock/biolo_datasets/variation/gnomAD/r2.1.1/vcf/exomes/gnomad.exomes.r2.1.1.sites.ht/"
# # gs = "/data/scratch/gnomAD/v3/gnomad.genomes.v3.1.sites.ht/"
# data = hl.read_table(gs)
# data

Running on Apache Spark version 2.4.1
SparkUI available at http://studio.local:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.64-1ef70187dc78
LOGGING: writing to /enadisk/maison/sage2016/weber/PycharmProjects/EXOTIC/notebooks/Hail/hail-20210602-1621-0.2.64-1ef70187dc78.log


In [171]:
bed_path = yaml['3_EXONS_PROPERTIES']['TMP']['refseq_bed']
mkdir(os.path.dirname(bed_path))

In [159]:
refseq_processed = pd.read_parquet(yaml['2_EXPRESSION']['Final']['refseq_corrected_cds_recomputed'])
refseq_processed = refseq_processed.loc[(refseq_processed['mRNA_nb_total'] > 1 ) & (refseq_processed['CDS_count'] > 1 )]
refseq_processed

biomart = pd.read_csv(yaml['1_GENOMICS']['External']['biomart'], compression='gzip', sep='\t')
refseq_processed = pd.merge(biomart[['Gene name', 'Chromosome/scaffold name']].drop_duplicates().rename({'Gene name' : 'Gene', 'Chromosome/scaffold name' : 'CHROM'}, axis=1), refseq_processed, on=['Gene'])
refseq_processed['CHROM'] = refseq_processed['CHROM'].astype(str)
refseq_processed['MAP'] = refseq_processed['Gene'] + '_' + refseq_processed['ranges']

refseq_processed = refseq_processed.loc[~refseq_processed['CHROM'].str.contains('HSCHR|HG')]
refseq_processed = refseq_processed[['CHROM', 'Start', 'End', 'MAP']].sort_values(by=['CHROM', 'Start'])
refseq_processed['Start'] = refseq_processed['Start'] - 1
refseq_processed['End'] = refseq_processed['End'] - 1
bed_path = yaml['3_EXONS_PROPERTIES']['TMP']['refseq_bed_gnomad']
mkdir(os.path.dirname(bed_path))
refseq_processed.to_csv(bed_path, header=False, sep='\t', index=False)

In [ ]:
refseq_processed

In [179]:
bed_file = hl.import_bed(bed_path, reference_genome='GRCh37')
bed_file.show()
# bed_file = bed_file.head(100)
# bed_file.show()

2021-05-07 10:22:59 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
  Loading field 'f3' as type str (user-supplied)


,
interval,target
interval<locus<GRCh37>>,str
[1:901912-1:901994),"""PLEKHN1_901912-901994"""
[1:902084-1:902183),"""PLEKHN1_902084-902183"""
[1:905657-1:905803),"""PLEKHN1_905657-905803"""
[1:905901-1:905981),"""PLEKHN1_905901-905981"""
[1:906066-1:906138),"""PLEKHN1_906066-906138"""
[1:906259-1:906386),"""PLEKHN1_906259-906386"""
[1:906457-1:906588),"""PLEKHN1_906457-906588"""
[1:906493-1:906588),"""PLEKHN1_906493-906588"""


In [196]:
ccrs_table.interval.end.show()

2021-05-07 10:31:13 Hail: INFO: Coerced sorted dataset


,
interval,
interval<locus<GRCh37>>,locus<GRCh37>
[1:901912-1:901913),1:901913
[1:901913-1:901914),1:901914
[1:901914-1:901920),1:901920
[1:901920-1:901921),1:901921
[1:901921-1:901922),1:901922
[1:901922-1:901923),1:901923
[1:901923-1:901924),1:901924
[1:901924-1:901927),1:901927


In [ ]:
ccrs_table = hl.import_bed('/gstock/EXOTIC/data/CONSERVATION/phylocsf_data_lite.bed', reference_genome='GRCh37')
# ccrs_table = ccrs_table.annotate(
#     start = ccrs_table.interval.start,
#     end = ccrs_table.interval.end,
# )
ccrs_table.show()

In [ ]:
bed_file.head(200).tail(50).show()

In [ ]:
ccrs_table = ccrs_table.annotate(
    MAP = bed_file[ccrs_table_lite.interval].target,
    Gene = hl.str(bed_file[ccrs_table_lite.interval].target).split('_')[0],
)

In [199]:
ccrs_table_lite = ccrs_table.filter(hl.is_defined(bed_file[ccrs_table.start]))
ccrs_table_lite = ccrs_table_lite.annotate(
    MAP = bed_file[ccrs_table_lite.start].target,
    Gene = hl.str(bed_file[ccrs_table_lite.start].target).split('_')[0],
)
ccrs_table_lite.show() 

2021-05-07 10:32:30 Hail: INFO: Coerced sorted dataset
2021-05-07 10:32:31 Hail: INFO: Coerced sorted dataset
2021-05-07 10:32:31 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-07 10:32:33 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-07 10:32:35 Hail: INFO: Coerced sorted dataset
2021-05-07 10:32:36 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-07 10:32:38 Hail: INFO: Coerced sorted dataset
2021-05-07 10:32:40 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-07 10:32:43 Hail: INFO: Coerced sorted dataset


,,,,,
interval,target,start,end,MAP,Gene
interval<locus<GRCh37>>,str,locus<GRCh37>,locus<GRCh37>,str,str
[1:901912-1:901913),"""0.749507173""",1:901912,1:901913,"""PLEKHN1_901912-901994""","""PLEKHN1"""
[1:901913-1:901914),"""0""",1:901913,1:901914,"""PLEKHN1_901912-901994""","""PLEKHN1"""
[1:901914-1:901920),"""26.62098847""",1:901914,1:901920,"""PLEKHN1_901912-901994""","""PLEKHN1"""
[1:901920-1:901921),"""0""",1:901920,1:901921,"""PLEKHN1_901912-901994""","""PLEKHN1"""
[1:901921-1:901922),"""0.749507173""",1:901921,1:901922,"""PLEKHN1_901912-901994""","""PLEKHN1"""
[1:901922-1:901923),"""0""",1:901922,1:901923,"""PLEKHN1_901912-901994""","""PLEKHN1"""
[1:901923-1:901924),"""0""",1:901923,1:901924,"""PLEKHN1_901912-901994""","""PLEKHN1"""
[1:901924-1:901927),"""9.178962193""",1:901924,1:901927,"""PLEKHN1_901912-901994""","""PLEKHN1"""


In [161]:
data_lite = data.filter(hl.is_defined(bed_file[data.locus]))
data_lite = data_lite.filter(data_lite.variant_type == 'snv')
data_lite = data_lite.filter(data_lite.freq[0]['AC'] > 0)
# data_lite['AF'] = data_lite.freq[0]['AC']
data_lite = data_lite.annotate(
    MAP = bed_file[data_lite.locus].target,
    AF = data_lite.freq[0]['AF'],
    id = hl.str(data_lite.vep.id).replace('/', '_'),
    Gene = hl.str(bed_file[data_lite.locus].target).split('_')[0],
)
data_lite = data_lite.select(data_lite.AF, data_lite.rsid, data_lite.variant_type, data_lite.vep.end, data_lite.id, data_lite.vep.most_severe_consequence, data_lite.MAP, data_lite.Gene)

# data_lite = data_lite.key_by('id')

In [ ]:
data_lite.describe()

In [ ]:
data_lite.filter(data_lite.Gene == 'MIB2').tail(20).show()

In [206]:
ccr_start = pd.read_parquet(yaml["3_EXONS_PROPERTIES"]["TMP"]["refseq_ccr_start"].replace('.tsv.gz', '.parquet'))
ccr_start

,interval,target,start,end,MAP,Gene
0,[1:901912-901913),0.749507,1:901912,1:901913,PLEKHN1_901912-901994,PLEKHN1
1,[1:901913-901914),0.000000,1:901913,1:901914,PLEKHN1_901912-901994,PLEKHN1
2,[1:901914-901920),26.620988,1:901914,1:901920,PLEKHN1_901912-901994,PLEKHN1
3,[1:901920-901921),0.000000,1:901920,1:901921,PLEKHN1_901912-901994,PLEKHN1
4,[1:901921-901922),0.749507,1:901921,1:901922,PLEKHN1_901912-901994,PLEKHN1
...,...,...,...,...,...,...
3641349,[X:154774917-154774918),0.571070,X:154774917,X:154774918,TMLHE_154774757-154774937,TMLHE
3641350,[X:154774918-154774919),0.000000,X:154774918,X:154774919,TMLHE_154774757-154774937,TMLHE
3641351,[X:154774919-154774932),29.559816,X:154774919,X:154774932,TMLHE_154774757-154774937,TMLHE
3641352,[X:154774932-154774933),0.000000,X:154774932,X:154774933,TMLHE_154774757-154774937,TMLHE


In [207]:
ccr_end = pd.read_parquet(yaml["3_EXONS_PROPERTIES"]["TMP"]["refseq_ccr_end"].replace('.tsv.gz', '.parquet'))
ccr_end

,interval,target,start,end,MAP,Gene
0,[1:901912-901913),0.749507,1:901912,1:901913,PLEKHN1_901912-901994,PLEKHN1
1,[1:901913-901914),0.000000,1:901913,1:901914,PLEKHN1_901912-901994,PLEKHN1
2,[1:901914-901920),26.620988,1:901914,1:901920,PLEKHN1_901912-901994,PLEKHN1
3,[1:901920-901921),0.000000,1:901920,1:901921,PLEKHN1_901912-901994,PLEKHN1
4,[1:901921-901922),0.749507,1:901921,1:901922,PLEKHN1_901912-901994,PLEKHN1
...,...,...,...,...,...,...
3554891,[X:154774915-154774917),0.000000,X:154774915,X:154774917,TMLHE_154774757-154774937,TMLHE
3554892,[X:154774917-154774918),0.571070,X:154774917,X:154774918,TMLHE_154774757-154774937,TMLHE
3554893,[X:154774918-154774919),0.000000,X:154774918,X:154774919,TMLHE_154774757-154774937,TMLHE
3554894,[X:154774919-154774932),29.559816,X:154774919,X:154774932,TMLHE_154774757-154774937,TMLHE


In [210]:
concat_start_end = pd.concat([
    pd.read_parquet(yaml["3_EXONS_PROPERTIES"]["TMP"]["refseq_ccr_start"].replace('.tsv.gz', '.parquet')),
    pd.read_parquet(yaml["3_EXONS_PROPERTIES"]["TMP"]["refseq_ccr_end"].replace('.tsv.gz', '.parquet')),
    ]
).sort_values(by='interval').drop_duplicates(subset='interval')
concat_start_end

,interval,target,start,end,MAP,Gene
1953387,[10:100003848-100003856),63.055177,10:100003848,10:100003856,R3HCC1L_100003848-100003915,R3HCC1L
1907619,[10:100003848-100003856),63.055177,10:100003848,10:100003856,R3HCC1L_100003848-100003915,R3HCC1L
1953388,[10:100003856-100003857),0.000000,10:100003856,10:100003857,R3HCC1L_100003848-100003915,R3HCC1L
1907620,[10:100003856-100003857),0.000000,10:100003856,10:100003857,R3HCC1L_100003848-100003915,R3HCC1L
1907621,[10:100003857-100003858),0.000000,10:100003857,10:100003858,R3HCC1L_100003848-100003915,R3HCC1L
...,...,...,...,...,...,...
3526307,[X:99956959-99957026),90.007142,X:99956959,X:99957026,SYTL4_99956924-99957033,SYTL4
3611489,[X:99956959-99957026),90.007142,X:99956959,X:99957026,SYTL4_99956924-99957033,SYTL4
3611490,[X:99957026-99957027),0.000000,X:99957026,X:99957027,SYTL4_99956924-99957033,SYTL4
3526308,[X:99957026-99957027),0.000000,X:99957026,X:99957027,SYTL4_99956924-99957033,SYTL4


In [203]:
concat_start_end.drop_duplicates()

,interval,target,start,end,MAP,Gene
0,[1:901912-901913),0.749507,1:901912,1:901913,PLEKHN1_901912-901994,PLEKHN1
1,[1:901913-901914),0.000000,1:901913,1:901914,PLEKHN1_901912-901994,PLEKHN1
2,[1:901914-901920),26.620988,1:901914,1:901920,PLEKHN1_901912-901994,PLEKHN1
3,[1:901920-901921),0.000000,1:901920,1:901921,PLEKHN1_901912-901994,PLEKHN1
4,[1:901921-901922),0.749507,1:901921,1:901922,PLEKHN1_901912-901994,PLEKHN1
...,...,...,...,...,...,...
3553249,[X:154002962-154002973),27.162020,X:154002962,X:154002973,DKC1_154002877-154002980,DKC1
3553419,[X:154014612-154014623),28.221109,X:154014612,X:154014623,MPP1_154014615-154014675,MPP1
3553469,[X:154020467-154020468),0.000000,X:154020467,X:154020468,MPP1_154020468-154020560,MPP1
3554608,[X:154301645-154301650),5.307590,X:154301645,X:154301650,BRCC3_154301649-154301706,BRCC3


In [212]:
concat_start_end.loc[concat_start_end['interval'].duplicated(keep=False) == False].target.describe()

count    90526.000000
mean        45.922442
std         32.896328
min          0.000000
25%         14.059613
50%         46.673487
75%         75.594818
max        100.000000
Name: target, dtype: float64

In [275]:
phylocsf_refseq = pd.read_parquet("/gstock/EXOTIC/data/EXON_PROPERTIES/refseq_phylocsf.parquet").rename({'target' : 'phyloCSF_score'}, axis=1)
phylocsf_refseq.loc[(phylocsf_refseq['phyloCSF_score'] <= -100), 'phyloCSF_cat'] = 'Unconserved'
phylocsf_refseq.loc[(phylocsf_refseq['phyloCSF_score'] > 1000), 'phyloCSF_cat'] = 'Conserved'
phylocsf_refseq.loc[(phylocsf_refseq['phyloCSF_score'] >= -100) & (phylocsf_refseq['phyloCSF_score'] < 1000), 'phyloCSF_cat'] = 'Standard'
phylocsf_refseq = phylocsf_refseq.fillna(False)
phylocsf_refseq

,interval,phyloCSF_score,start,end,MAP,Gene,phyloCSF_cat
57629,[10:100177932-100178014),444.0738,10:100177932,10:100178014,HPS1_100177932-100178014,HPS1,Standard
57630,[10:100179802-100179915),326.2875,10:100179802,10:100179915,HPS1_100179802-100179915,HPS1,Standard
57631,[10:100182126-100182270),786.1477,10:100182126,10:100182270,HPS1_100182126-100182270,HPS1,Standard
57632,[10:100183360-100183425),162.4925,10:100183360,10:100183425,HPS1_100183360-100183425,HPS1,Standard
57633,[10:100183365-100183425),158.0360,10:100183365,10:100183425,HPS1_100183360-100183425,HPS1,Standard
...,...,...,...,...,...,...,...
109992,[Y:2829132-2829687),1290.2635,Y:2829132,Y:2829687,ZFY_2829115-2829687,ZFY,Conserved
109993,[Y:2843136-2843285),358.4581,Y:2843136,Y:2843285,ZFY_2843136-2843285,ZFY,Standard
109994,[Y:2843552-2843695),400.0841,Y:2843552,Y:2843695,ZFY_2843552-2843695,ZFY,Standard
109995,[Y:2844711-2844863),800.6360,Y:2844711,Y:2844863,ZFY_2844711-2844863,ZFY,Standard


In [237]:
refseq_exons = pd.read_parquet('/gstock/EXOTIC/data/GENOMICS/refseq_37_corrected_recomputed.parquet')
refseq_exons['MAP'] = refseq_exons['Gene'] + '_' + refseq_exons['ranges']
refseq_exons

,Gene,ranges,mRNA_exons,mRNA_nb,Share,Strand,mRNA_gene,mRNA_nb_total,Ratio,Ratio_num,Const_Alt,Ratio_num_bins,Start,End,Length,CDS_count,MAP
0,A2M,9220419-9220435,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,Const,0.8 - 1,9220419,9220435,16,38,A2M_9220419-9220435
1,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,Const,0.8 - 1,9220779,9220820,41,38,A2M_9220779-9220820
2,A2M,9221336-9221438,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,Const,0.8 - 1,9221336,9221438,102,38,A2M_9221336-9221438
3,A2M,9222341-9222409,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,Const,0.8 - 1,9222341,9222409,68,38,A2M_9222341-9222409
4,A2M,9223084-9223174,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,Const,0.8 - 1,9223084,9223174,90,38,A2M_9223084-9223174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178476,ZZZ3,78047461-78047576,"[NM_001308237, NM_015534]",2,False,0,"[NM_001308237, NM_015534]",2,2/2,1.0,Const,0.8 - 1,78047461,78047576,115,12,ZZZ3_78047461-78047576
178477,ZZZ3,78047664-78047811,"[NM_001308237, NM_015534]",2,False,0,"[NM_001308237, NM_015534]",2,2/2,1.0,Const,0.8 - 1,78047664,78047811,147,12,ZZZ3_78047664-78047811
178478,ZZZ3,78050202-78050340,"[NM_001308237, NM_015534]",2,False,0,"[NM_001308237, NM_015534]",2,2/2,1.0,Const,0.8 - 1,78050202,78050340,138,12,ZZZ3_78050202-78050340
178479,ZZZ3,78097535-78099039,[NM_015534],1,False,0,"[NM_001308237, NM_015534]",2,1/2,0.5,Alt,0.4 - 0.6,78097535,78099039,1504,12,ZZZ3_78097535-78099039


In [303]:
phylocsf_refseq = pd.read_parquet("/gstock/EXOTIC/data/EXON_PROPERTIES/refseq_phylocsf.parquet").rename({'target' : 'phyloCSF_score'}, axis=1)
phylocsf_refseq.loc[(phylocsf_refseq['phyloCSF_score'] <= -100), 'phyloCSF_cat'] = 'Unconserved'
phylocsf_refseq.loc[(phylocsf_refseq['phyloCSF_score'] > 1000), 'phyloCSF_cat'] = 'Conserved'
phylocsf_refseq.loc[(phylocsf_refseq['phyloCSF_score'] >= -100) & (phylocsf_refseq['phyloCSF_score'] < 1000), 'phyloCSF_cat'] = 'Standard'
phylocsf_refseq = phylocsf_refseq.fillna(False)

m = pd.merge(refseq_exons, phylocsf_refseq, on='MAP')
m = m.loc[m['phyloCSF_cat'] != 'Standard']
m


l = list()
for col in ['phyloCSF_cat']:
#     for unique in m[col].unique().tolist():
    print(m[['Const_Alt', cat]].pivot(columns='Const_Alt', values=cat).apply(pd.Series.value_counts))
t = m[['Const_Alt', cat]].pivot(columns='Const_Alt', values=cat).apply(pd.Series.value_counts).drop([np.nan], axis=1)
t = t.values
print(t)
odd_ratio = (t[0][1]/t[1][1]) / (t[0][0] / t[1][0])
se = np.sqrt((1/t[0][1]) + (1/t[1][1]) + (1/(t[0][0]) + (1/t[1][0])))
lower = np.exp(np.log(odd_ratio) - 1.96 *se)
upper = np.exp(np.log(odd_ratio) + 1.96 *se)
print(t, odd_ratio, se, lower, upper)
l.append(
    {
        'Cat' : cat,
        'OR' : odd_ratio,
        'SE' : se,
        'Lower_CI' : lower,
        'Upper_CI' : upper,
    }
)
l
            



Const_Alt    NaN   Alt  Const
Conserved      2  1711   4733
Unconserved    1  1797   1741
[[1711 4733]
 [1797 1741]]
[[1711 4733]
 [1797 1741]] 2.855195174246714 0.043893069916154215 2.619830932193564 3.1117044168250967


[{'Cat': 'phyloCSF_cat',
  'OR': 2.855195174246714,
  'SE': 0.043893069916154215,
  'Lower_CI': 2.619830932193564,
  'Upper_CI': 3.1117044168250967}]

In [305]:
gnomad = pd.read_parquet('/gstock/EXOTIC/data/EXON_PROPERTIES/refseq_miso_gnomad_variations.parquet')
gnomad

,locus,alleles,AF,rsid,variant_type,end,id,most_severe_consequence,MAP,Gene
0,1:901913,"[""T"",""G""]","1,2295e-05",rs759148200,snv,901913,1_901913_T_G,start_lost,PLEKHN1_901912-901994,PLEKHN1
1,1:901916,"[""G"",""A""]","4,0919e-06",rs764917347,snv,901916,1_901916_G_A,missense_variant,PLEKHN1_901912-901994,PLEKHN1
2,1:901917,"[""G"",""A""]","4,0947e-06",rs774961129,snv,901917,1_901917_G_A,synonymous_variant,PLEKHN1_901912-901994,PLEKHN1
3,1:901919,"[""A"",""C""]","2,2641e-04",rs763342668,snv,901919,1_901919_A_C,missense_variant,PLEKHN1_901912-901994,PLEKHN1
4,1:901920,"[""C"",""G""]","4,0754e-06",rs1408404600,snv,901920,1_901920_C_G,missense_variant,PLEKHN1_901912-901994,PLEKHN1
...,...,...,...,...,...,...,...,...,...,...
2810848,Y:22749975,"[""C"",""T""]","1,7257e-05",rs746588234,snv,22749975,Y_22749975_C_T,synonymous_variant,EIF1AY_22749910-22749991,EIF1AY
2810849,Y:22751395,"[""T"",""C""]","1,4826e-05",rs1382594808,snv,22751395,Y_22751395_T_C,synonymous_variant,EIF1AY_22751370-22751461,EIF1AY
2810850,Y:22751399,"[""C"",""G""]","1,4843e-05",rs762602059,snv,22751399,Y_22751399_C_G,missense_variant,EIF1AY_22751370-22751461,EIF1AY
2810851,Y:22751428,"[""C"",""T""]","2,9772e-05",rs1158365540,snv,22751428,Y_22751428_C_T,synonymous_variant,EIF1AY_22751370-22751461,EIF1AY


In [310]:
clinvar.loc[clinvar['locus.position'].isna() == True]

,locus.contig,locus.position,alleles,rsid,ALLELEID,CLNREVSTAT,CLNSIG,GENEINFO,RS,MAP,Gene,RS_STARS


In [312]:
clinvar = pd.read_parquet('/gstock/EXOTIC/data/EXON_PROPERTIES/refseq_clinvar_pathogenic.parquet')
clinvar['id'] = clinvar['locus.contig'] + '_' + clinvar['locus.position'].astype(str) + '_' + clinvar['alleles'].apply(lambda r: "_".join(r))
clinvar

,locus.contig,locus.position,alleles,rsid,ALLELEID,CLNREVSTAT,CLNSIG,GENEINFO,RS,MAP,Gene,RS_STARS,id
71,1,1289293,"[A, T]",402162,389106,"criteria_provided,_single_submitter",Likely_pathogenic,MXRA8:54587,[374879755],MXRA8_1289228-1289308,MXRA8,1,1_1289293_A_T
72,1,1372327,"[C, T]",830326,818588,no_assertion_criteria_provided,Likely_pathogenic,VWA1:64856,None,VWA1_1372307-1372864,VWA1,0,1_1372327_C_T
82,1,1451416,"[T, G]",828051,816386,no_assertion_criteria_provided,Pathogenic,ATAD3A:55210,[1570319915],ATAD3A_1451392-1451468,ATAD3A,0,1_1451416_T_G
89,1,1454346,"[C, T]",828050,816385,no_assertion_criteria_provided,Pathogenic,ATAD3A:55210,[760826883],ATAD3A_1454301-1454370,ATAD3A,0,1_1454346_C_T
101,1,1460622,"[T, G]",637018,624841,"criteria_provided,_single_submitter",Pathogenic,ATAD3A:55210,[1570345942],ATAD3A_1460620-1460671,ATAD3A,1,1_1460622_T_G
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256897,X,154250724,"[T, C]",10352,25391,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852476],F8_154250685-154250827,F8,0,X_154250724_T_C
256899,X,154250739,"[T, A]",10154,25193,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852378],F8_154250685-154250827,F8,0,X_154250739_T_A
256900,X,154250751,"[A, C]",10153,25192,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852377],F8_154250685-154250827,F8,0,X_154250751_A_C
256902,X,154250785,"[G, A]",10152,25191,"criteria_provided,_single_submitter",Pathogenic,F8:2157,[387906432],F8_154250685-154250827,F8,1,X_154250785_G_A


In [325]:
merge_variants = pd.concat(
    [
        gnomad.loc[~gnomad['id'].isin(clinvar.id.values.tolist())],
        clinvar.loc[~clinvar['id'].isin(gnomad.id.values.tolist())]
    ]
)
merge_variants

,locus,alleles,AF,rsid,variant_type,end,id,most_severe_consequence,MAP,Gene,locus.contig,locus.position,ALLELEID,CLNREVSTAT,CLNSIG,GENEINFO,RS,RS_STARS
0,1:901913,"[""T"",""G""]","1,2295e-05",rs759148200,snv,901913.0,1_901913_T_G,start_lost,PLEKHN1_901912-901994,PLEKHN1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1:901916,"[""G"",""A""]","4,0919e-06",rs764917347,snv,901916.0,1_901916_G_A,missense_variant,PLEKHN1_901912-901994,PLEKHN1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1:901917,"[""G"",""A""]","4,0947e-06",rs774961129,snv,901917.0,1_901917_G_A,synonymous_variant,PLEKHN1_901912-901994,PLEKHN1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1:901919,"[""A"",""C""]","2,2641e-04",rs763342668,snv,901919.0,1_901919_A_C,missense_variant,PLEKHN1_901912-901994,PLEKHN1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1:901920,"[""C"",""G""]","4,0754e-06",rs1408404600,snv,901920.0,1_901920_C_G,missense_variant,PLEKHN1_901912-901994,PLEKHN1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256896,NaN,"[C, A]",NaN,10156,NaN,NaN,X_154250707_C_A,NaN,F8_154250685-154250827,F8,X,154250707.0,25195.0,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852379],0.0
256897,NaN,"[T, C]",NaN,10352,NaN,NaN,X_154250724_T_C,NaN,F8_154250685-154250827,F8,X,154250724.0,25391.0,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852476],0.0
256899,NaN,"[T, A]",NaN,10154,NaN,NaN,X_154250739_T_A,NaN,F8_154250685-154250827,F8,X,154250739.0,25193.0,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852378],0.0
256900,NaN,"[A, C]",NaN,10153,NaN,NaN,X_154250751_A_C,NaN,F8_154250685-154250827,F8,X,154250751.0,25192.0,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852377],0.0


In [384]:
# phylocsf_refseq = pd.read_parquet("/gstock/EXOTIC/data/EXON_PROPERTIES/refseq_phylocsf.parquet").rename({'target' : 'phyloCSF_score'}, axis=1)
# phylocsf_refseq.loc[(phylocsf_refseq['phyloCSF_score'] <= -100), 'phyloCSF_cat'] = 'Unconserved'
# phylocsf_refseq.loc[(phylocsf_refseq['phyloCSF_score'] > 1000), 'phyloCSF_cat'] = 'Conserved'
# phylocsf_refseq.loc[(phylocsf_refseq['phyloCSF_score'] >= -100) & (phylocsf_refseq['phyloCSF_score'] < 1000), 'phyloCSF_cat'] = 'Standard'
# phylocsf_refseq = phylocsf_refseq.fillna(False)

# m = pd.merge(refseq_exons, phylocsf_refseq, on='MAP')
# m = m.loc[m['phyloCSF_cat'] != 'Standard']
# m


def stats(cat_const, cat_alt, others_const, others_alt, col, cat, ):
    odd_ratio = (cat_const/others_const) / (cat_alt /others_alt)
    se = np.sqrt((1/cat_const) + (1/others_const) + (1/cat_alt) + (1/others_alt))
    lower = np.exp(np.log(odd_ratio) - 1.96 *se)
    upper = np.exp(np.log(odd_ratio) + 1.96 *se)
    print(t, odd_ratio, se, lower, upper)
    d = {
            'col' : col,
            'cat' : cat,
            'OR' : odd_ratio,
            'SE' : se,
            'Lower_CI' : lower,
            'Upper_CI' : upper,
        }
    return d

def compute_odds_ratio(df, cols):
    l = list()
    for col in cols:
        print(col)
        print(df[['Const_Alt', col]].pivot(columns='Const_Alt', values=col).apply(pd.Series.value_counts))
        t = df[['Const_Alt', col]].pivot(columns='Const_Alt', values=col).apply(pd.Series.value_counts).fillna(0)
        print(t)
        for cat in df[col].unique().tolist():
            print("Cat : ", cat)
            cat_const = t.loc[cat]['Const']
            cat_alt = t.loc[cat]['Alt']
            others_const = t.loc[~t.index.isin([cat])]['Const'].sum()
            others_alt = t.loc[~t.index.isin([cat])]['Alt'].sum()
            l.append(stats(cat_const, cat_alt, others_const, others_alt, col, cat, ))
    return l
 


In [392]:
[1/(10 ** e) for e in list(range(0,6,1))]

[1.0, 0.1, 0.01, 0.001, 0.0001, 1e-05]

In [397]:
m_lite

,Gene_x,ranges,mRNA_exons,mRNA_nb,Share,Strand,mRNA_gene,mRNA_nb_total,Ratio,Ratio_num,...,End,Length,CDS_count,MAP,interval,target,start,end,Gene_y,CCR_bins
0,A2M,9220419-9220435,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,9220435,16,38,A2M_9220419-9220435,[12:9220419-9220422),94.960308,12:9220419,12:9220422,A2M,90 - 95
1,A2M,9220419-9220435,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,9220435,16,38,A2M_9220419-9220435,[12:9220422-9220436),94.960308,12:9220422,12:9220436,A2M,90 - 95
2,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,9220820,41,38,A2M_9220779-9220820,[12:9220779-9220793),94.960308,12:9220779,12:9220793,A2M,90 - 95
3,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,9220820,41,38,A2M_9220779-9220820,[12:9220793-9220794),0.000000,12:9220793,12:9220794,A2M,0 - 20
4,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,9220820,41,38,A2M_9220779-9220820,[12:9220794-9220811),72.870475,12:9220794,12:9220811,A2M,20 - 80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,ABCA8,66864442-66864497,"[NM_001288986, NM_007168, NM_001288985]",3,False,0,"[NM_007168, NM_001288986, NM_001288985]",3,3/3,1.0,...,66864497,55,39,ABCA8_66864442-66864497,[17:66864452-66864453),0.000000,17:66864452,17:66864453,ABCA8,0 - 20
9996,ABCA8,66864442-66864497,"[NM_001288986, NM_007168, NM_001288985]",3,False,0,"[NM_007168, NM_001288986, NM_001288985]",3,3/3,1.0,...,66864497,55,39,ABCA8_66864442-66864497,[17:66864453-66864458),23.536853,17:66864453,17:66864458,ABCA8,20 - 80
9997,ABCA8,66864442-66864497,"[NM_001288986, NM_007168, NM_001288985]",3,False,0,"[NM_007168, NM_001288986, NM_001288985]",3,3/3,1.0,...,66864497,55,39,ABCA8_66864442-66864497,[17:66864458-66864459),0.000000,17:66864458,17:66864459,ABCA8,0 - 20
9998,ABCA8,66864442-66864497,"[NM_001288986, NM_007168, NM_001288985]",3,False,0,"[NM_007168, NM_001288986, NM_001288985]",3,3/3,1.0,...,66864497,55,39,ABCA8_66864442-66864497,[17:66864459-66864461),6.586176,17:66864459,17:66864461,ABCA8,0 - 20


In [401]:
m_lite

,Gene_x,ranges,mRNA_exons,mRNA_nb,Share,Strand,mRNA_gene,mRNA_nb_total,Ratio,Ratio_num,...,MAP,locus,alleles,AF,rsid,variant_type,end,id,most_severe_consequence,Gene_y
0,A2M,9220419-9220435,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,A2M_9220419-9220435,12:9220425,"[""A"",""G""]","4,0480e-06",rs1350615491,snv,9220425,12_9220425_A_G,synonymous_variant,A2M
1,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,A2M_9220779-9220820,12:9220787,"[""C"",""T""]","4,0337e-06",rs770612196,snv,9220787,12_9220787_C_T,missense_variant,A2M
2,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,A2M_9220779-9220820,12:9220793,"[""G"",""A""]","4,0456e-06",rs776462123,snv,9220793,12_9220793_G_A,missense_variant,A2M
3,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,A2M_9220779-9220820,12:9220801,"[""C"",""T""]","8,0997e-06",rs1226282307,snv,9220801,12_9220801_C_T,synonymous_variant,A2M
4,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,A2M_9220779-9220820,12:9220811,"[""G"",""T""]","4,0913e-06",rs1280952806,snv,9220811,12_9220811_G_T,missense_variant,A2M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,ABCB6,220081374-220081554,"[NM_001349828, NM_005689]",2,False,0,"[NM_001349828, NM_005689]",2,2/2,1.0,...,ABCB6_220081374-220081554,2:220081415,"[""C"",""T""]","2,4199e-04",rs200125320,snv,220081415,2_220081415_C_T,missense_variant,ABCB6
9996,ABCB6,220081374-220081554,"[NM_001349828, NM_005689]",2,False,0,"[NM_001349828, NM_005689]",2,2/2,1.0,...,ABCB6_220081374-220081554,2:220081416,"[""G"",""A""]","8,7828e-03",rs57467915,snv,220081416,2_220081416_G_A,missense_variant,ABCB6
9997,ABCB6,220081374-220081554,"[NM_001349828, NM_005689]",2,False,0,"[NM_001349828, NM_005689]",2,2/2,1.0,...,ABCB6_220081374-220081554,2:220081424,"[""C"",""T""]","4,6228e-06",rs1225134091,snv,220081424,2_220081424_C_T,missense_variant,ABCB6
9998,ABCB6,220081374-220081554,"[NM_001349828, NM_005689]",2,False,0,"[NM_001349828, NM_005689]",2,2/2,1.0,...,ABCB6_220081374-220081554,2:220081429,"[""G"",""C""]","5,8475e-05",rs550618866,snv,220081429,2_220081429_G_C,synonymous_variant,ABCB6


In [402]:
# m = pd.merge(refseq_exons, gnomad, on='MAP')
# m_lite = m.head(10000)
bins_af = list(reversed([1/(10 ** e) for e in list(range(0,7,1))]))

def convert_bins_into_labels(bins):
    return [" - ".join([str(bins[j]), str(bins[j+1])]) for j, e in enumerate(bins) if j <len(bins)-1]
# convert_bins_into_labels(bins_af)
m_lite['AF'] = m_lite['AF'].str.replace(',', '.').astype(float)
m_lite['AF_bins'] = pd.cut(m_lite['AF'], bins=bins_af, labels=convert_bins_into_labels(bins_af))
m_lite
# m_lite.Const_Alt.value_counts()


/home/weber/.conda/envs/hail/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Gene_x,ranges,mRNA_exons,mRNA_nb,Share,Strand,mRNA_gene,mRNA_nb_total,Ratio,Ratio_num,...,locus,alleles,AF,rsid,variant_type,end,id,most_severe_consequence,Gene_y,AF_bins
0,A2M,9220419-9220435,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,12:9220425,"[""A"",""G""]",0.000004,rs1350615491,snv,9220425,12_9220425_A_G,synonymous_variant,A2M,1e-06 - 1e-05
1,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,12:9220787,"[""C"",""T""]",0.000004,rs770612196,snv,9220787,12_9220787_C_T,missense_variant,A2M,1e-06 - 1e-05
2,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,12:9220793,"[""G"",""A""]",0.000004,rs776462123,snv,9220793,12_9220793_G_A,missense_variant,A2M,1e-06 - 1e-05
3,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,12:9220801,"[""C"",""T""]",0.000008,rs1226282307,snv,9220801,12_9220801_C_T,synonymous_variant,A2M,1e-06 - 1e-05
4,A2M,9220779-9220820,"[NM_001347423, NM_001347425, NM_000014, NM_001...",4,False,0,"[NM_001347423, NM_001347424, NM_000014, NM_001...",4,4/4,1.0,...,12:9220811,"[""G"",""T""]",0.000004,rs1280952806,snv,9220811,12_9220811_G_T,missense_variant,A2M,1e-06 - 1e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,ABCB6,220081374-220081554,"[NM_001349828, NM_005689]",2,False,0,"[NM_001349828, NM_005689]",2,2/2,1.0,...,2:220081415,"[""C"",""T""]",0.000242,rs200125320,snv,220081415,2_220081415_C_T,missense_variant,ABCB6,0.0001 - 0.001
9996,ABCB6,220081374-220081554,"[NM_001349828, NM_005689]",2,False,0,"[NM_001349828, NM_005689]",2,2/2,1.0,...,2:220081416,"[""G"",""A""]",0.008783,rs57467915,snv,220081416,2_220081416_G_A,missense_variant,ABCB6,0.001 - 0.01
9997,ABCB6,220081374-220081554,"[NM_001349828, NM_005689]",2,False,0,"[NM_001349828, NM_005689]",2,2/2,1.0,...,2:220081424,"[""C"",""T""]",0.000005,rs1225134091,snv,220081424,2_220081424_C_T,missense_variant,ABCB6,1e-06 - 1e-05
9998,ABCB6,220081374-220081554,"[NM_001349828, NM_005689]",2,False,0,"[NM_001349828, NM_005689]",2,2/2,1.0,...,2:220081429,"[""G"",""C""]",0.000058,rs550618866,snv,220081429,2_220081429_G_C,synonymous_variant,ABCB6,1e-05 - 0.0001


In [362]:
m_lite.most_severe_consequence.value_counts()

missense_variant           6542
synonymous_variant         3134
stop_gained                 193
splice_region_variant        99
start_lost                   14
stop_lost                    12
stop_retained_variant         3
splice_acceptor_variant       2
5_prime_UTR_variant           1
Name: most_severe_consequence, dtype: int64

In [404]:
compute_odds_ratio(m_lite, ['most_severe_consequence', 'AF_bins'])

most_severe_consequence
Const_Alt                  Alt   Const
5_prime_UTR_variant        1.0     NaN
missense_variant         695.0  5847.0
splice_acceptor_variant    NaN     2.0
splice_region_variant     10.0    89.0
start_lost                 6.0     8.0
stop_gained               24.0   169.0
stop_lost                  5.0     7.0
stop_retained_variant      1.0     2.0
synonymous_variant       306.0  2828.0
Const_Alt                  Alt   Const
5_prime_UTR_variant        1.0     0.0
missense_variant         695.0  5847.0
splice_acceptor_variant    0.0     2.0
splice_region_variant     10.0    89.0
start_lost                 6.0     8.0
stop_gained               24.0   169.0
stop_lost                  5.0     7.0
stop_retained_variant      1.0     2.0
synonymous_variant       306.0  2828.0
Cat :  synonymous_variant
Const_Alt     Alt  Const
Conserved    1711   4733
Unconserved  1797   1741 1.1197645180432285 0.07164203728519647 0.9730692948288098 1.2885748040063016
Cat :  missense_va

/home/weber/.conda/envs/hail/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
/home/weber/.conda/envs/hail/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
/home/weber/.conda/envs/hail/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
/home/weber/.conda/envs/hail/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
/home/weber/.conda/envs/hail/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


[{'col': 'most_severe_consequence',
  'cat': 'synonymous_variant',
  'OR': 1.1197645180432285,
  'SE': 0.07164203728519647,
  'Lower_CI': 0.9730692948288098,
  'Upper_CI': 1.2885748040063016},
 {'col': 'most_severe_consequence',
  'cat': 'missense_variant',
  'OR': 0.9564480589441491,
  'SE': 0.06902752479870228,
  'Lower_CI': 0.8354182816677471,
  'Upper_CI': 1.0950118156761277},
 {'col': 'most_severe_consequence',
  'cat': 'splice_region_variant',
  'OR': 1.0423332957237956,
  'SE': 0.3351300863938027,
  'Lower_CI': 0.5404283486629323,
  'Upper_CI': 2.010365855274663},
 {'col': 'most_severe_consequence',
  'cat': 'stop_gained',
  'OR': 0.8209799233367491,
  'SE': 0.22062240421886634,
  'Lower_CI': 0.5327641031403509,
  'Upper_CI': 1.2651153306859608},
 {'col': 'most_severe_consequence',
  'cat': 'stop_lost',
  'OR': 0.16324203465623252,
  'SE': 0.5864535019304489,
  'Lower_CI': 0.05171692569806736,
  'Upper_CI': 0.5152657765135187},
 {'col': 'most_severe_consequence',
  'cat': 'start

In [376]:
bins = [0,20,80,90,95,100]

[" - ".join([str(bins[j]), str(bins[j+1])]) for j, e in enumerate(bins) if j <len(bins)-1]

['0 - 20', '20 - 80', '80 - 90', '90 - 95', '95 - 100']

In [378]:
ccr = pd.read_parquet('/gstock/EXOTIC/data/EXON_PROPERTIES/refseq_ccr_start_end.parquet')
bins = [0,20,80,90,95,100]
ccr['CCR_bins'] = pd.cut(ccr['target'], bins, labels=[" - ".join([str(bins[j]), str(bins[j+1])]) for j, e in enumerate(bins) if j <len(bins)-1], include_lowest=True)
ccr

,interval,target,start,end,MAP,Gene,CCR_bins
1953387,[10:100003848-100003856),63.055177,10:100003848,10:100003856,R3HCC1L_100003848-100003915,R3HCC1L,20 - 80
1953388,[10:100003856-100003857),0.000000,10:100003856,10:100003857,R3HCC1L_100003848-100003915,R3HCC1L,0 - 20
1907621,[10:100003857-100003858),0.000000,10:100003857,10:100003858,R3HCC1L_100003848-100003915,R3HCC1L,0 - 20
1953390,[10:100003858-100003874),71.921144,10:100003858,10:100003874,R3HCC1L_100003848-100003915,R3HCC1L,20 - 80
1907623,[10:100003874-100003875),0.000000,10:100003874,10:100003875,R3HCC1L_100003848-100003915,R3HCC1L,0 - 20
...,...,...,...,...,...,...,...
3526305,[X:99956953-99956958),8.932019,X:99956953,X:99956958,SYTL4_99956924-99957033,SYTL4,0 - 20
3526306,[X:99956958-99956959),0.000000,X:99956958,X:99956959,SYTL4_99956924-99957033,SYTL4,0 - 20
3526307,[X:99956959-99957026),90.007142,X:99956959,X:99957026,SYTL4_99956924-99957033,SYTL4,90 - 95
3611490,[X:99957026-99957027),0.000000,X:99957026,X:99957027,SYTL4_99956924-99957033,SYTL4,0 - 20


In [380]:
m = pd.merge(refseq_exons, ccr, on='MAP')
m_lite = m.head(10000)
m_lite.Const_Alt.value_counts()


Const    8464
Alt      1536
Name: Const_Alt, dtype: int64

In [383]:
compute_odds_ratio(m, ['CCR_bins'])

CCR_bins
Const_Alt  NaN     Alt    Const
0 - 20     343  825820  1980838
20 - 80     73  208281   530896
80 - 90      1   14777    42923
90 - 95      1    5457    17548
95 - 100     2    3629    12799
Const_Alt  NaN     Alt    Const
0 - 20     343  825820  1980838
20 - 80     73  208281   530896
80 - 90      1   14777    42923
90 - 95      1    5457    17548
95 - 100     2    3629    12799
Cat :  90 - 95
Const_Alt     Alt  Const
Conserved    1711   4733
Unconserved  1797   1741 1.3182435507075838 0.015542748298482092 1.2786903888190908 1.3590201929859016
Cat :  0 - 20
Const_Alt     Alt  Const
Conserved    1711   4733
Unconserved  1797   1741 0.9216472770607997 0.002771028656663815 0.9166551803190102 0.9266665607213068
Cat :  20 - 80
Const_Alt     Alt  Const
Conserved    1711   4733
Unconserved  1797   1741 1.0543710046285748 0.0028893862631653488 1.0484167697546907 1.0603590551700026
Cat :  95 - 100
Const_Alt     Alt  Const
Conserved    1711   4733
Unconserved  1797   1741 1.4456465245

[{'Cat': '90 - 95',
  'OR': 1.3182435507075838,
  'SE': 0.015542748298482092,
  'Lower_CI': 1.2786903888190908,
  'Upper_CI': 1.3590201929859016},
 {'Cat': '0 - 20',
  'OR': 0.9216472770607997,
  'SE': 0.002771028656663815,
  'Lower_CI': 0.9166551803190102,
  'Upper_CI': 0.9266665607213068},
 {'Cat': '20 - 80',
  'OR': 1.0543710046285748,
  'SE': 0.0028893862631653488,
  'Lower_CI': 1.0484167697546907,
  'Upper_CI': 1.0603590551700026},
 {'Cat': '95 - 100',
  'OR': 1.4456465245939014,
  'SE': 0.018842142808086198,
  'Lower_CI': 1.3932317429632792,
  'Upper_CI': 1.5000332031090595},
 {'Cat': '80 - 90',
  'OR': 1.1920008817965122,
  'SE': 0.00960844664059193,
  'Lower_CI': 1.1697625189950471,
  'Upper_CI': 1.214662018256783}]

In [ ]:
m = pd.merge(refseq_exons, phylocsf_refseq, on='MAP')

In [317]:
merge_variants.CLNSIG.value_counts()

Benign                                         2805750
Pathogenic                                       14737
Likely_pathogenic                                 8048
Pathogenic/Likely_pathogenic                      1586
Pathogenic,_other                                   11
Pathogenic,_risk_factor                              5
Pathogenic,_association                              5
Pathogenic/Likely_pathogenic,_other                  3
Likely_pathogenic,_other                             2
Pathogenic/Likely_pathogenic,_drug_response          2
Likely_pathogenic,_drug_response                     2
Likely_pathogenic,_risk_factor                       1
Name: CLNSIG, dtype: int64

In [324]:
merge_variants

,locus,alleles,AF,rsid,variant_type,end,id,most_severe_consequence,MAP,Gene,locus.contig,locus.position,ALLELEID,CLNREVSTAT,CLNSIG,GENEINFO,RS,RS_STARS,Status
0,1:901913,"[""T"",""G""]","1,2295e-05",rs759148200,snv,901913.0,1_901913_T_G,start_lost,PLEKHN1_901912-901994,PLEKHN1,NaN,NaN,NaN,NaN,Benign,NaN,NaN,NaN,Pathogenic
1,1:901916,"[""G"",""A""]","4,0919e-06",rs764917347,snv,901916.0,1_901916_G_A,missense_variant,PLEKHN1_901912-901994,PLEKHN1,NaN,NaN,NaN,NaN,Benign,NaN,NaN,NaN,Pathogenic
2,1:901917,"[""G"",""A""]","4,0947e-06",rs774961129,snv,901917.0,1_901917_G_A,synonymous_variant,PLEKHN1_901912-901994,PLEKHN1,NaN,NaN,NaN,NaN,Benign,NaN,NaN,NaN,Pathogenic
3,1:901919,"[""A"",""C""]","2,2641e-04",rs763342668,snv,901919.0,1_901919_A_C,missense_variant,PLEKHN1_901912-901994,PLEKHN1,NaN,NaN,NaN,NaN,Benign,NaN,NaN,NaN,Pathogenic
4,1:901920,"[""C"",""G""]","4,0754e-06",rs1408404600,snv,901920.0,1_901920_C_G,missense_variant,PLEKHN1_901912-901994,PLEKHN1,NaN,NaN,NaN,NaN,Benign,NaN,NaN,NaN,Pathogenic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256896,NaN,"[C, A]",NaN,10156,NaN,NaN,X_154250707_C_A,NaN,F8_154250685-154250827,F8,X,154250707.0,25195.0,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852379],0.0,Pathogenic
256897,NaN,"[T, C]",NaN,10352,NaN,NaN,X_154250724_T_C,NaN,F8_154250685-154250827,F8,X,154250724.0,25391.0,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852476],0.0,Pathogenic
256899,NaN,"[T, A]",NaN,10154,NaN,NaN,X_154250739_T_A,NaN,F8_154250685-154250827,F8,X,154250739.0,25193.0,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852378],0.0,Pathogenic
256900,NaN,"[A, C]",NaN,10153,NaN,NaN,X_154250751_A_C,NaN,F8_154250685-154250827,F8,X,154250751.0,25192.0,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852377],0.0,Pathogenic


In [329]:
gnomad = pd.read_parquet('/gstock/EXOTIC/data/EXON_PROPERTIES/refseq_miso_gnomad_variations.parquet')
clinvar = pd.read_parquet('/gstock/EXOTIC/data/EXON_PROPERTIES/refseq_clinvar_pathogenic.parquet')
clinvar['id'] = clinvar['locus.contig'] + '_' + clinvar['locus.position'].astype(str) + '_' + clinvar['alleles'].apply(lambda r: "_".join(r))
merge_variants = pd.concat(
    [
        gnomad.loc[~gnomad['id'].isin(clinvar.id.values.tolist())],
        clinvar.loc[~clinvar['id'].isin(gnomad.id.values.tolist())]
    ]
)
merge_variants.loc[merge_variants['CLNSIG'].isna() == True, 'Status'] = 'Benign'
merge_variants['Status'] = merge_variants['Status'].fillna('Pathogenic')

m = pd.merge(refseq_exons, merge_variants, on='MAP')

l = list()
for cat in ['Status']:
#     for unique in m[col].unique().tolist():
    print(m[['Const_Alt', cat]].pivot(columns='Const_Alt', values=cat).apply(pd.Series.value_counts))
t = m[['Const_Alt', cat]].pivot(columns='Const_Alt', values=cat).apply(pd.Series.value_counts).drop([np.nan], axis=1)
t = t.values
print(t)
odd_ratio = (t[0][0] / t[1][0]) / (t[0][1]/t[1][1])
se = np.sqrt((1/t[0][1]) + (1/t[1][1]) + (1/(t[0][0]) + (1/t[1][0])))
lower = np.exp(np.log(odd_ratio) - 1.96 *se)
upper = np.exp(np.log(odd_ratio) + 1.96 *se)
print(t, odd_ratio, se, lower, upper)
l.append(
    {
        'Cat' : cat,
        'OR' : odd_ratio,
        'SE' : se,
        'Lower_CI' : lower,
        'Upper_CI' : upper,
    }
)
l
            



Const_Alt   NaN     Alt    Const
Benign      283  819814  1985653
Pathogenic    2    6646    17754
[[ 819814 1985653]
 [   6646   17754]]
[[ 819814 1985653]
 [   6646   17754]] 1.1029297537386094 0.01444005480123557 1.0721516790447403 1.134591369353391


[{'Cat': 'Status',
  'OR': 1.1029297537386094,
  'SE': 0.01444005480123557,
  'Lower_CI': 1.0721516790447403,
  'Upper_CI': 1.134591369353391}]

In [16]:
clinvar = hl.read_matrix_table("/gstock/biolo_datasets/variation/variation_sets/clinvar/vcf_GRCh37/v2/clinvar_20210123.ht")
clinvar = clinvar.filter_rows(clinvar.info.CLNVC == "single_nucleotide_variant")
clinvar = clinvar.filter_rows(hl.str(clinvar.info.CLNSIG).contains("athogenic"))
clinvar = clinvar.filter_rows(~hl.str(clinvar.info.CLNSIG).contains("onflict"))

# clinvar_lite = clinvar.select_rows(
#     clinvar.rsid,
#     clinvar.info.ALLELEID,
#     clinvar.info.CLNREVSTAT,
#     clinvar.info.CLNSIG,
#     clinvar.info.GENEINFO,
#     clinvar.info.RS,
#     clinvar.info.CLNVCSO,
# )
clinvar.rows().head(20).show()

+---------------+------------+----------+-----------+----------+-------------+--------------+
| locus         | alleles    | rsid     |      qual | filters  | info.AF_ESP | info.AF_EXAC |
+---------------+------------+----------+-----------+----------+-------------+--------------+
| locus<GRCh37> | array<str> | str      |   float64 | set<str> |     float64 |      float64 |
+---------------+------------+----------+-----------+----------+-------------+--------------+
| 1:879375      | ["C","T"]  | "950448" | -1,00e+01 | NA       |          NA |           NA |
| 1:899892      | ["C","A"]  | "916564" | -1,00e+01 | NA       |          NA |           NA |
| 1:949523      | ["C","T"]  | "183381" | -1,00e+01 | NA       |          NA |           NA |
| 1:949739      | ["G","T"]  | "161454" | -1,00e+01 | NA       |          NA |     1,00e-05 |
| 1:957605      | ["G","A"]  | "243036" | -1,00e+01 | NA       |          NA |     1,00e-05 |
| 1:957693      | ["A","T"]  | "243037" | -1,00e+01 | NA       |          NA |           NA |
| 1:976962      | ["C","T"]  | "126556" | -1,00e+01 | NA       |          NA |           NA |
| 1:977433      | ["C","G"]  | "644955" | -1,00e+01 | NA       |          NA |           NA |
| 1:980867      | ["C","T"]  | "950399" | -1,00e+01 | NA       |          NA |           NA |
| 1:985052      | ["C","T"]  | "489335" | -1,00e+01 | NA       |          NA |           NA |
| 1:985853      | ["G","A"]  | "243039" | -1,00e+01 | NA       |          NA |     3,00e-05 |
| 1:985955      | ["G","C"]  | "18241"  | -1,00e+01 | NA       |          NA |           NA |
| 1:986143      | ["G","T"]  | "126555" | -1,00e+01 | NA       |          NA |           NA |
| 1:986655      | ["T","C"]  | "617537" | -1,00e+01 | NA       |          NA |           NA |
| 1:1149118     | ["G","A"]  | "96692"  | -1,00e+01 | NA       |          NA |     3,00e-05 |
| 1:1167659     | ["A","G"]  | "60484"  | -1,00e+01 | NA       |          NA |           NA |
| 1:1167674     | ["C","T"]  | "60493"  | -1,00e+01 | NA       |          NA |           NA |
| 1:1167851     | ["A","G"]  | "60488"  | -1,00e+01 | NA       |          NA |           NA |
| 1:1167858     | ["C","T"]  | "60489"  | -1,00e+01 | NA       |          NA |           NA |
| 1:1167897     | ["G","A"]  | "452846" | -1,00e+01 | NA       |          NA |           NA |
+---------------+------------+----------+-----------+----------+-------------+--------------+

+-------------+---------------+
| info.AF_TGP | info.ALLELEID |
+-------------+---------------+
|     float64 |         int32 |
+-------------+---------------+
|          NA |        929884 |
|          NA |        904889 |
|          NA |        181485 |
|          NA |        171288 |
|          NA |        244110 |
|          NA |        244111 |
|          NA |        132069 |
|          NA |        626510 |
|          NA |        921548 |
|          NA |        481556 |
|          NA |        244113 |
|          NA |         33280 |
|          NA |        132068 |
|          NA |        608899 |
|          NA |        102585 |
|          NA |         75079 |
|          NA |         75088 |
|          NA |         75083 |
|          NA |         75084 |
|          NA |        442612 |
+-------------+---------------+

+-----------------------------------------------------------------------------------------------+
| info.CLNDN                                                                                    |
+-----------------------------------------------------------------------------------------------+
| array<str>                                                                                    |
+-----------------------------------------------------------------------------------------------+
| ["not_provided"]                                                                              |
| ["Tracheoesophageal_fistula"]                                                                 |
| ["Immunodeficiency_38_with_basal_g

In [6]:
clinvar.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AF_ESP: float64, 
        AF_EXAC: float64, 
        AF_TGP: float64, 
        ALLELEID: int32, 
        CLNDN: array<str>, 
        CLNDNINCL: array<str>, 
        CLNDISDB: array<str>, 
        CLNDISDBINCL: array<str>, 
        CLNHGVS: array<str>, 
        CLNREVSTAT: array<str>, 
        CLNSIG: array<str>, 
        CLNSIGCONF: array<str>, 
        CLNSIGINCL: array<str>, 
        CLNVC: str, 
        CLNVCSO: str, 
        CLNVI: array<str>, 
        DBVARID: array<str>, 
        GENEINFO: str, 
        MC: array<str>, 
        ORIGIN: array<str>, 
        RS: array<str>, 
        SSR: int32
    }
----------------------------------------
Entry fields:
    N